In [ ]:
# Cell 5: transform to numeric arrays and compute SHAP using TreeExplainer on the tree estimator
import shap
import numpy as np

# Ensure expected feature order
feature_cols = ["age", "tenure_months", "monthly_charges",
                "num_support_tickets", "contract_type", "payment_method", "has_internet"]

X_train = X_train.loc[:, feature_cols]
X_mis_raw = X_mis_raw.loc[:, feature_cols]

# Get pipeline pieces
prep = model.named_steps.get("prep", None)
clf = model.named_steps.get("clf", None)
if prep is None or clf is None:
    raise RuntimeError("Expected pipeline with steps named 'prep' and 'clf'.")

# Transform to numeric arrays (OneHotEncoder output may be sparse -> convert)
X_train_trans = prep.transform(X_train)
X_mis_trans = prep.transform(X_mis_raw)

if hasattr(X_train_trans, "toarray"):
    X_train_trans = X_train_trans.toarray()
if hasattr(X_mis_trans, "toarray"):
    X_mis_trans = X_mis_trans.toarray()

print("X_train_trans.shape:", X_train_trans.shape)
print("X_mis_trans.shape:", X_mis_trans.shape)

# Use TreeExplainer on the trained tree estimator (RandomForest)
#Learners to type



# For multiclass RandomForest, shap_values will be a list; for binary it may return array
sv = explainer.shap_values(X_mis_trans)

# Normalize to shap_matrix (n_samples, n_transformed_features)
if isinstance(sv, list):
    # pick class 1 (positive) contributions if present
    class_idx = 1 if len(sv) > 1 else 0
    shap_matrix = np.asarray(sv[class_idx])
else:
    shap_matrix = np.asarray(sv)

print("shap_matrix.shape:", shap_matrix.shape)
n_trans_features = X_train_trans.shape[1]
if shap_matrix.shape[1] != n_trans_features:
    raise ValueError(f"SHAP feature dim {shap_matrix.shape[1]} != expected {n_trans_features}")

# Derive transformed feature names for plotting
try:
    trans_feature_names = prep.get_feature_names_out()
except Exception:
    numeric_features = ["age","tenure_months","monthly_charges","num_support_tickets","has_internet"]
    cat_features = ["contract_type","payment_method"]
    cat_names = prep.named_transformers_["cat"].get_feature_names_out(cat_features)
    trans_feature_names = list(numeric_features) + list(cat_names)

print("Number of transformed features:", len(trans_feature_names))
